In [44]:
import pandas as pd
import numpy as np
import tensorflow as TF
from typing import Tuple, List
import cv2
import os
from sklearn.model_selection import train_test_split
CATEGORIES = [0, 1, 2, 3, 4, 5]
EPOCHS = 25
TEST = 0.3

In [45]:
def load_signs_dataset(dataDir : str): # type: ignore
    images = []
    labels = []
    for SingleCategory in CATEGORIES:
        category = os.path.join(dataDir, str(SingleCategory))
        
        for image in os.listdir(category):
            img_path = os.path.join(category, image)
            finalImage = cv2.imread(img_path)
            
            if finalImage is not None:
                img = cv2.resize(finalImage, (64, 64))
                images.append(img)
                labels.append(SingleCategory)
    
    return (np.array(images), np.array(labels))
                

In [46]:
images, labels = load_signs_dataset("DataSet/archive")

X_train_orig, Y_train_orig, X_test_orig, Y_test_orig= train_test_split(
    np.array(images), np.array(labels), test_size=TEST, train_size=1-TEST
)

print(X_train_orig.shape)
print(Y_train_orig.shape)
print(X_test_orig.shape)
print(Y_test_orig.shape)

(865, 64, 64, 3)
(372, 64, 64, 3)
(865,)
(372,)


In [ ]:
def model(input_shape):
    input_img = TF.keras.Input(shape=input_shape)
    Z1 = TF.keras.layers.Conv2D(filters=8, kernel_size=(4,4), padding="same")(input_img)
    
    A1 = TF.keras.layers.ReLU()(Z1)
    
    P1 = TF.keras.layers.MaxPool2D(pool_size=(8,8), strides=8, padding="same")(A1)
    
    Z2 = TF.keras.layers.Conv2D(filters=16, kernel_size=(2,2), strides=1, padding="same")(P1)
    
    A2 = TF.keras.layers.ReLU()(Z2)
    
    P2 = TF.keras.layers.MaxPool2D(pool_size=(4,4), strides=4, padding="same")(A2)
    
    
    F = TF.keras.layers.Flatten()(P2)
    
    outpus = TF.keras.layers.Dense(units= 12, activation="softmax")(F)
    
    model = TF.keras.Model(input_img,outpus)
    return model